In [26]:
#! Install library
import os
import importlib.util

if importlib.util.find_spec("requests") is None:
    os.system("pip install requests")

if importlib.util.find_spec("beautifulsoup4") is None:
    os.system("pip install beautifulsoup4")

if importlib.util.find_spec("Counter") is None:
    os.system("pip install Counter")

if importlib.util.find_spec("openpyxl") is None:
    os.system("pip install openpyxl")

In [27]:
Debug = False

In [7]:
#! Authentication details
import os
import dotenv
import importlib.util

JiraHost = None
ConfluenceHost = None
Username = None
Password = None

if importlib.util.find_spec("google.colab") is not None:  ## if using google colab
    if not os.path.exists(".env"):
        from google.colab import files

        uploaded = files.upload()
        file_name = list(uploaded.keys())[0]
        try:
            os.rename(file_name, ".env")
        except:
            pass

try:
    dotenv.load_dotenv("../.env", override=True)

    JiraHost = os.getenv("SECRETS_HOST")
    ConfluenceHost = os.getenv("SECRETS_CONFLUENCE")
    Username = os.getenv("SECRETS_USERNAME")
    Password = os.getenv("SECRETS_PASSWORD")
except:
    display("trouble loading dot env")
    pass

if JiraHost is None or JiraHost == "":
    JiraHost = input("Enter Jira Host")

if ConfluenceHost is None or ConfluenceHost == "":
    ConfluenceHost = input("Enter Confluence Host")

if Username is None or Username == "":
    Username = input("Enter Username")

if Password is None or Password == "":
    Password = input("Enter Password")

display("Jira Host: " + JiraHost)
display("Confluence Host: " + ConfluenceHost)

'Jira Host: https://autoandgeneral-sandbox-377.atlassian.net/'

'Confluence Host: https://autoandgeneral-sandbox-377.atlassian.net/wiki/'

In [5]:
#! Functions
import base64
import re
from bs4 import BeautifulSoup
import requests
from collections import Counter
import pandas as pd
import requests
from functools import reduce
import warnings
import json

requests.packages.urllib3.disable_warnings(
    requests.packages.urllib3.exceptions.InsecureRequestWarning
)


def _ExpandColumn(
    self: pd.DataFrame,
    colName: str,
    columnsToExpand=[],
    prefix: str = "Prefix",
    sentenceCase: bool = True,
) -> pd.DataFrame:
    if prefix == "Prefix":
        prefix = colName + " "
        with warnings.catch_warnings():
            warnings.simplefilter(action="ignore", category=FutureWarning)
            expandedCols = self[colName].apply(
                lambda x: pd.Series(x).add_prefix(prefix)
            )
        columnsToExpand = [prefix + c for c in columnsToExpand]
    else:
        expandedCols = self[colName].apply(lambda x: pd.Series(x))

    if len(columnsToExpand) > 0:
        expandedCols = expandedCols[columnsToExpand]

    if sentenceCase:
        expandedCols.columns = [fnSentenceCase(c) for c in expandedCols.columns]

    return pd.concat([self.drop(colName, axis=1), expandedCols], axis=1)


pd.DataFrame.expand = _ExpandColumn


def fnSentenceCase(s):
    s = " ".join(dict.fromkeys(s.split()))  # remove duplicate words
    s = s.replace("0", "")  # remove "0"
    s = s.strip()
    return " ".join(
        [x.capitalize() for x in re.sub(r"([A-Z])", r" \1", s).split()]
    )  # sentence case


def _SentenceCaseColumns(self: pd.DataFrame) -> pd.DataFrame:
    self.columns = [fnSentenceCase(c) for c in self.columns]
    return self


pd.DataFrame.sentence_case_columns = _SentenceCaseColumns


def flatten_reduce_lambda(frm):
    try:
        return list(reduce(lambda x, y: x + y, frm, []))
    except:
        return list(reduce(lambda x, y: x + y, [frm], []))


def most_frequent(List):
    try:
        c = Counter(List)
        most_common = [key for key, _ in c.most_common(5)]
        return most_common
    except:
        return ""


def fnGetDefaultHeaders():
    return {
        "content-type": "application/json",
        "authorization": "Basic "
        + base64.b64encode((Username + ":" + Password).encode()).decode(),
        "retry-after": "120",
    }


def fnUpdateBoardAdmins(boardId, boardAdmins, boardGroups):
    boardAdminsData = {}
    boardAdminsData["userKeys"] = boardAdmins
    boardAdminsData["groupKeys"] = boardGroups
    data = {}
    data["id"] = boardId
    data["boardAdmins"] = boardAdminsData
    url = "/rest/greenhopper/1.0/rapidviewconfig/boardadmins"
    headers = fnGetDefaultHeaders()
    response = requests.put(
        JiraHost + url, headers=headers, data=json.dumps(data), verify=False
    )
    return response.json()


def fnUpdateFilterOwner(filterId, jql, name, description, filterOwner):
    data = {}
    data["jql"] = jql
    data["name"] = name
    data["description"] = description
    data["owner"] = {}
    data["owner"]["key"] = filterOwner
    url = "/rest/api/2/filter/" + str(filterId)
    headers = fnGetDefaultHeaders()
    response = requests.put(
        JiraHost + url, headers=headers, data=json.dumps(data), verify=False
    )
    return response.json()


def fnUpdateFilterOwnerAndShare(filterId, jql, name, description, filterOwner):
    permission = {}
    permission["type"] = "authenticated"
    permission["view"] = "true"
    #permission["edit"] = "true"

    # permission["type"] = "group"
    # permission["group"] = {}
    # permission["group"]["name"] = "jira-users"
    data = permission
    url = "/rest/api/2/filter/" + str(filterId) + "/permission"
    headers = fnGetDefaultHeaders()
    response = requests.post(
        JiraHost + url, headers=headers, data=json.dumps(data), verify=False
    )
    return response.json()


def fnGetFilterDetails(filterId):
    url = "/rest/api/2/filter/" + str(filterId)
    headers = fnGetDefaultHeaders()
    response = requests.get(JiraHost + url, headers=headers, verify=False)
    return response.json()


def fnBoards():
    def ApiCall(startAt):
        url = "/rest/greenhopper/1.0/rapidviews/viewsData"
        headers = fnGetDefaultHeaders()
        defaultContents = {"startAt": startAt}
        response = requests.get(
            JiraHost + url, headers=headers, json=defaultContents, verify=False
        )
        return response.json()

    values = fnAPI(ApiCall)
    return values


def fnUsers():
    # /rest/api/2/user/search?username=.&includeInactive=true&maxResults=1000&startAt=7000
    def ApiCall(startAt):
        url = (
            "/rest/api/2/user/search?username=.&includeInactive=true&maxResults=1000&startAt="
            + str(startAt)
        )
        headers = fnGetDefaultHeaders()
        response = requests.get(JiraHost + url, headers=headers, verify=False)
        return response.json()

    return pd.DataFrame(flatten_reduce_lambda([ApiCall(i * 1000) for i in range(8)]))


def fnAPI(webRequestDelegate, startAt=0) -> pd.DataFrame:
    def innerGetResults(webRequestDelegate, startAt=0):
        results = webRequestDelegate(startAt)
        if isinstance(results, dict) and "total" in results and "maxResults" in results:
            if startAt + results["maxResults"] < results["total"]:
                return [results] + innerGetResults(
                    webRequestDelegate, startAt + results["maxResults"]
                )
            else:
                return [results]
        else:
            return [results]

    Source = flatten_reduce_lambda(innerGetResults(webRequestDelegate, startAt))
    df = pd.DataFrame(Source)
    return df

In [8]:
users = fnUsers()

display(users.head())

,errorMessages,errors
0,[The query parameter 'username' is not support...,{}
1,[The query parameter 'username' is not support...,{}
2,[The query parameter 'username' is not support...,{}
3,[The query parameter 'username' is not support...,{}
4,[The query parameter 'username' is not support...,{}


In [9]:
df = fnBoards()

df = df.explode("views")
df = df.drop(["total", "globalConfig"], axis=1)
df = df.expand("views", [], None, False)
df = df.drop(["canEdit", "sprintSupportEnabled"], axis=1)
df = df.expand("filter", [], "Prefix", False)
df = df.drop(
    [
        "filter canEdit",
        "filter isOrderedByRank",
        "filter permissionEntries",
        "filter canBeFixed",
    ],
    axis=1,
)
df = df.expand("filter owner", [], "Prefix", False)
df = df.drop(["filter owner renderedLink"], axis=1)

df2 = df.expand("boardAdmins", [], "Prefix", False)
df2 = df2.expand("boardAdmins userKeys", [], "Prefix", False)
for i in range(6):
    df2 = df2.expand("boardAdmins userKeys " + str(i), [], "Prefix", False)

df2 = df2.expand("boardAdmins groupKeys", [], "Prefix", False)
for i in range(2):
    df2 = df2.expand("boardAdmins groupKeys " + str(i), [], "Prefix", False)

try:
    df2["boardAdmins groups"] = df2[
        ["boardAdmins groupKeys " + str(i) + " key" for i in range(1)]
    ].apply(
        lambda row: "|".join(
            filter(lambda value: str(value).upper() != "NAN", row.values.astype(str))
        ),
        axis=1,
    )
    df["boardAdmin Groups"] = df2["boardAdmins groups"]
except:
    pass


def linkRel(row):
    soup = BeautifulSoup(" ".join(row.values.astype(str)), "html.parser").findAll("a")
    return (
        "|".join(a.get("rel")[0] if a != None else "" for a in soup)
        if soup != None
        else ""
    )


cols = ["boardAdmins userKeys " + str(i) + " displayName" for i in range(6)]
df["boardAdminUsernames"] = df2[cols].apply(linkRel, axis=1)


df["newAdmins"] = ""
df["newFilterOwner"] = ""
df = df.reset_index()  # make sure indexes pair with number of rows

## This is not correct pandas but i couldn't make it work otherwise!
for index, row in df.iterrows():
    newAdmins = []
    requiresDefault = False
    for username in row["boardAdminUsernames"].split("|"):
        userRecord = users.loc[users["name"] == username]
        if userRecord.empty:
            continue
        if userRecord["active"].bool():
            newAdmins.append(username)
    if len(newAdmins) == 0:  ## & requiresDefault:
        df.at[index, "newAdmins"] = "pogliani"
    else:
        df.at[index, "newAdmins"] = "|".join(newAdmins)

    filterOwner = row["filter owner userName"]
    userRecord = users.loc[users["name"] == filterOwner]
    if userRecord.empty:
        continue
    if userRecord["active"].bool():
        df.at[index, "newFilterOwner"] = filterOwner
    else:
        df.at[index, "newFilterOwner"] = "pogliani"

df["requiresUpdateToBoard"] = df["newAdmins"] != df["boardAdminUsernames"]
df["requiresUpdateToFilter"] = df["newFilterOwner"] != df["filter owner userName"]

# df.to_excel('../ListBoardOwners.xlsx', index=False)

display(df)

KeyError: "['total'] not found in axis"

In [34]:
#! Remove Inactive Board Users

removeInactiveBoardUsersDf = df[df["requiresUpdateToBoard"]]
removeInactiveBoardUsersDf = removeInactiveBoardUsersDf[
    removeInactiveBoardUsersDf["requiresUpdateToBoard"]
]
removeInactiveBoardUsersDf = removeInactiveBoardUsersDf.drop(
    [
        "boardAdmins",
        "name",
        "filter id",
        "filter owner userName",
        "filter name",
        "filter query",
        "filter owner displayName",
        "newFilterOwner",
        "requiresUpdateToBoard",
        "requiresUpdateToFilter",
    ],
    axis=1,
)

display(removeInactiveBoardUsersDf)

errors = 0
updated = 0
for index, row in removeInactiveBoardUsersDf.iterrows():
    admins = []
    for username in row["newAdmins"].split("|"):
        userRecord = users.loc[users["name"] == username]
        admins.append(userRecord["key"].values[0])

    try:
        print(
            (
                row["id"],
                admins,
                row["boardAdmin Groups"].split("|")
                if len(row["boardAdmin Groups"]) > 0
                else [],
            )
        )

        result = (
            {"errorMessages": "debug mode"}
            if Debug
            else fnUpdateBoardAdmins(
                row["id"],
                admins,
                row["boardAdmin Groups"].split("|")
                if len(row["boardAdmin Groups"]) > 0
                else [],
            )
        )
        if "errorMessages" in result:
            print(
                (
                    result,
                    row["id"],
                    admins,
                    row["boardAdmin Groups"].split("|")
                    if len(row["boardAdmin Groups"]) > 0
                    else [],
                )
            )
            errors = errors + 1
        else:
            updated = updated + 1

    except Exception as ex:
        print(ex, (row["id"], admins))
        errors = errors + 1

if errors > 0:
    print(f"Errors {errors} out of {len(removeInactiveBoardUsersDf.index)}")
else:
    if updated > 0:
        print(f"Updated {updated} out of {len(removeInactiveBoardUsersDf.index)}")
    else:
        print("Clean!")

,index,id,boardAdminUsernames,newAdmins
501,0,1043,smigliori|chardcastl,smigliori


'boardAdmin Groups' (1043, ['smigliori'])
Errors 1 out of 1


In [35]:
#! Update Filter Users
import json

changeFilterOwner = df[df["requiresUpdateToFilter"]]
#changeFilterOwner = changeFilterOwner[changeFilterOwner["filter id"].isin([38941])]
changeFilterOwner = changeFilterOwner.drop(
    [
        "boardAdmins",
        "name",
        "id",
        "boardAdminUsernames",
        "requiresUpdateToBoard",
        "newAdmins",
        "requiresUpdateToFilter",
    ],
    axis=1,
)
try:
    changeFilterOwner = changeFilterOwner.drop(["boardAdmin Groups"], axis=1)
except:
    pass

display(changeFilterOwner)

errors = 0
updated = 0
for index, row in changeFilterOwner.iterrows():
    filter = {}
    try:
        filter = fnGetFilterDetails(row["filter id"])
    except Exception as ex:
        print(
            "Error getting filter details",
            ex,
            (
                row["filter id"],
                row["filter query"],
                row["filter name"],
                row["filter owner userName"],
                row["newFilterOwner"],
            ),
        )
        errors = errors + 1
        continue

    try:
        description = str(filter["description"])
    except:
        description = ""

    try:
        result = (
            {"errorMessages": "debug mode"}
            if Debug
            else fnUpdateFilterOwner(
                row["filter id"],
                row["filter query"],
                row["filter name"],
                description,
                row["newFilterOwner"],
            )
        )
        if "errorMessages" in result:
            try:
                for message in result["errorMessages"]:
                    if (message == "Dashboards and filters in this Jira instance cannot be shared with anyone on the web anymore. If you want to modify this item, you must change the Shares so that it is not shared with anyone on the web."):
                        print("updating share")
                        result = (
                            {"errorMessages": "debug mode"}
                            if Debug
                            else fnUpdateFilterOwnerAndShare(
                                row["filter id"],
                                row["filter query"],
                                row["filter name"],
                                description,
                                row["newFilterOwner"],
                            )
                        )
                        print(result)
                        if not "errorMessages" in result:
                            updated = updated + 1
                            continue
                else:
                    pass
            except:
                pass

            print(
                result,
                (
                    row["filter id"],
                    row["filter query"],
                    row["filter name"],
                    description,
                    row["filter owner userName"],
                    row["newFilterOwner"],
                ),
            )
            errors = errors + 1
        else:
            updated = updated + 1
    except Exception as ex:
        print(
            "Error updating",
            ex,
            (
                row["filter id"],
                row["filter query"],
                row["filter name"],
                description,
                row["filter owner userName"],
                row["newFilterOwner"],
            ),
        )
        errors = errors + 1

if errors > 0:
    print(f"Errors {errors} out of {len(changeFilterOwner.index)}")

if updated > 0:
    print(f"Updated {updated} out of {len(changeFilterOwner.index)}")
else:
    if errors == 0:
        print("Clean!")

,index,filter id,filter name,filter query,filter owner userName,filter owner displayName,newFilterOwner
4,0,33183,Africa_promo_filter,labels = Africa_promo_initiative,ijohnson,Isaac Johnson [X],pogliani
5,0,37433,Filter for AGGI Spirit Board,project = AGQL ORDER BY Rank ASC,ghari,Govind Hari [X],pogliani
7,0,39081,Filter for Change Delivery Board,project = CHDLV ORDER BY Rank ASC,ghari,Govind Hari [X],pogliani
8,0,44493,Filter for System Support - AGP Initiatives,project = AGP AND resolution = Unresolved AND ...,rewalsh,Reece Walsh [X],pogliani
9,0,38941,AGP: Large or Complex,"project = AGP AND ""Portfolio Change Type"" = ""L...",mhastie,Melody Hastie [X],pogliani
...,...,...,...,...,...,...,...
867,0,41385,Filter for TT board,project = TT ORDER BY Rank ASC,mhastie,Melody Hastie [X],pogliani
868,0,41399,Filter for TT Sprint 1,project = TT ORDER BY Rank ASC,gboon,Gareth Boon [X],pogliani
871,0,42553,Filter for UAT Defect Board,project = UATT,fhowell,Fiona Howell [X],pogliani
872,0,42553,Filter for UAT Defect Board,project = UATT,fhowell,Fiona Howell [X],pogliani


updating share
[{'id': 63489, 'type': 'loggedin', 'view': True, 'edit': False}]
[{'id': 63489, 'type': 'loggedin', 'view': True, 'edit': False}] (33183, 'labels = Africa_promo_initiative', 'Africa_promo_filter', '', 'ijohnson', 'pogliani')
updating share
[{'id': 63493, 'type': 'loggedin', 'view': True, 'edit': False}]
[{'id': 63493, 'type': 'loggedin', 'view': True, 'edit': False}] (38941, 'project = AGP AND "Portfolio Change Type" = "Large or Complex" ORDER BY Rank ASC', 'AGP: Large or Complex', '', 'mhastie', 'pogliani')
updating share
[{'id': 63494, 'type': 'loggedin', 'view': True, 'edit': False}]
[{'id': 63494, 'type': 'loggedin', 'view': True, 'edit': False}] (38960, 'project = AGP AND fixVersion in (MIAP, "COVID-19 Digital Acceleration", "COVID-19 Business Response") ORDER BY Rank ASC', 'AGP Board:MIAP-Covid19', '', 'mhastie', 'pogliani')
updating share
[{'id': 63495, 'type': 'loggedin', 'view': True, 'edit': False}]
[{'id': 63495, 'type': 'loggedin', 'view': True, 'edit': False